In [36]:
from bs4 import BeautifulSoup
import requests
import re
import requests
import pandas as pd

def search_company_page(company):
    company_page = ""
    #Given a company, find its webpage URL through the research system of Reuter

    #This URL return the search page for the given company
    search_url = f'https://www.reuters.com/search/news?sortBy=&dateRange=&blob={company}'

    #Creating url request and bs4 object
    req=requests.get(search_url)

    soup = BeautifulSoup(req.content, "html.parser")
    company_page = soup.find("div",class_="search-stock-ticker").findNext()["href"]

    #we return the final url of the webpage of the company
    return re.sub(r'overview','financial-highlights',"https://www.reuters.com"+company_page)

def get_data(company):
    url=search_company_page(company)
    req=requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    ##Get stock price from company page
    try:
        price_L=(soup.
               find("div",class_="sectionQuote nasdaqChange").
               findAll("span"))
        
        price = float(price_L[1].text.strip())
        currency=price_L[2].text.strip()
        
        
    except:
        price = "?"
        currency= "?"

    ##Get %change from company page
    try:
        change = float(re.sub(r'^[(]|[%)]+$',"",
                          soup.
                          find("div",class_="sectionQuote priceChange").
                          findAll("span")[4].
                          contents[0].
                          strip()))/100
        if change == "--":
            change = 0
            
    except:
        change = 0
        
        
    try:
    ##Get Shares owned by institutionnal holders
        shared_owned, =(float(re.sub(r'^[(]|[%)]+$',"",elmt.parent.parent.find(class_="data").contents[0]))
                        for elmt in soup.findAll("strong") 
                        if "Shares Owned" in elmt.contents[0])
    except:
        shared_owned = ""

    
    ##Get divident yield for company/industry/sector
    try:
        yield_list, *rest = (elmt.parent.findAll(class_="data")
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        pass
    
    try:
        company_divident_yield = float(yield_list[0].text)
    except:
        company_divident_yield = "?"
        
    try:    
        industry_divident_yield = float(yield_list[1].text)
    except:
        industry_divident_yield = "?"

    try:    
        sector_divident_yield = float(yield_list[2].text)
    except:
        sector_divident_yield = "?"
    
    try:
        sell_year_ending_list, *rest = (elmt.parent.findAll(class_="data")
                                       for elmt in soup.findAll("td") 
                                       if "Quarter Ending" in elmt.get_text())
        try:
            sell_year_ending_nr = int(sell_year_ending_list[0].text)
        except:
            sell_year_ending_nr = "?"

        try:
            sell_year_ending_mean = float(re.sub(r"[,]","",(sell_year_ending_list[1].text)))
        except:
            sell_year_ending_mean = "?"

        try:
            sell_year_ending_high = float(re.sub(r"[,]","",(sell_year_ending_list[2].text)))
        except:
            sell_year_ending_high = "?"

        try:
            sell_year_ending_low = float(re.sub(r"[,]","",(sell_year_ending_list[3].text)))
        except:
            sell_year_ending_low = "?"

    except:
        sell_year_ending_nr = "?"
        sell_year_ending_mean = "?"
        sell_year_ending_high = "?"
        sell_year_ending_low = "?"   
        
        
    try:
        market = soup.find("span",class_="nasdaqChangeHeader").text.split("on ")[1]
    except:
            marker = "?"
    


    return {"Company":company.upper(),
            "Marker":market,
            "Date":pd.Timestamp.today(tz="UTC").date(),
            "Price":price,
            "Currency":currency,
            "Change":change,
            "Shared_owned":shared_owned,
            "Company_divident_yield":company_divident_yield,
            "Industry_divident_yield":industry_divident_yield,
            "Sell_year_ending_nr":sell_year_ending_nr,
            "Sell_year_ending_mean":sell_year_ending_mean,
            "Sell_year_ending_high":sell_year_ending_high,
            "Sell_year_ending_low":sell_year_ending_low
           }
        

In [37]:
urlCAC = "https://www.boursier.com/indices/composition/cac-40-FR0003500008,FR.html?tri=dcapi"
reqCAC=requests.get(urlCAC)
soupCAC = BeautifulSoup(reqCAC.content, "html.parser")
CAC40 = [elmt["title"] if "(" not in elmt["title"] else elmt["title"].split("(")[0] for elmt in soupCAC.findAll("a", class_="name--wrapper")]

In [38]:
reuter_L = []
company_list = ["Airbus","Danone","LVMH","total","edf","michelin","carrefour",'valeo','renault','peugeot','technipfmc']
for company in CAC40:
    try:
        reuter_L.append(get_data(re.sub(r" GROUP","",company)))
    except:
        print("Fail "+company)

reuter_df = pd.DataFrame(reuter_L,columns=reuter_L[0].keys())

Fail PSA GROUPE 
Fail ACCORHOTELS


In [39]:
reuter_df.sort_values("Change",ascending=False)

,Company,Marker,Date,Price,Currency,Change,Shared_owned,Company_divident_yield,Industry_divident_yield,Sell_year_ending_nr,Sell_year_ending_mean,Sell_year_ending_high,Sell_year_ending_low
6,AXA,Oslo Stock Exchange,2018-10-12,23.19,NOK,0.0425,28.43,?,3.10,3,615.31,645.23,595.56
8,KERING,Paris Stock Exchange,2018-10-12,385.1,EUR,0.0344,29.37,1.43,0.71,5,4499.62,6752.1,3619
4,AIRBUS,Paris Stock Exchange,2018-10-12,100.22,EUR,0.0297,43.61,1.66,1.48,5,23493,26073.4,21431
37,VALEO,Paris Stock Exchange,2018-10-12,31.73,EUR,0.0242,54.67,3.9,1.87,1,5271,5271,5271
22,ARCELORMITTAL,Amsterdam Stock Exchange,2018-10-12,24.77,EUR,0.0236,25.91,0.33,2.60,4,18738.2,19892,17145.3
14,SCHNEIDER ELECTRIC,Paris Stock Exchange,2018-10-12,63.2,EUR,0.0223,48.18,3.36,1.05,3,6971.67,7072,6878
10,SAFRAN,Paris Stock Exchange,2018-10-12,108.8,EUR,0.0216,33.47,1.4,1.34,?,?,?,?
32,CARREFOUR,Paris Stock Exchange,2018-10-12,15.95,EUR,0.0179,23.64,4.12,1.76,1,22028,22028,22028
33,PUBLICIS,Paris Stock Exchange,2018-10-12,52.06,EUR,0.0176,46.08,3.81,1.26,?,?,?,?
24,RENAULT,Paris Stock Exchange,2018-10-12,71.86,EUR,0.0171,29.42,4.94,2.88,?,?,?,?


SyntaxError: unexpected EOF while parsing (<ipython-input-35-4ec785a2cc07>, line 12)